In [ ]:
!pip install -q cupy-cuda12x      

In [ ]:
'''
This is the PDE solver. Run this with the desired parameters and initial conditions.
The wave equations and EoM get solved using a RK4 scheme, with in the end a table of
the droplet's trajectory and snapshots of the wave on the free surface.
'''
# ----------------------- Dependencies -------------------------------------
import numpy as np        
import cupy as xp                  
import cupy.fft as cfft              
import matplotlib.pyplot as plt
from pathlib import Path

# -------------------- numerical grid  -------------------
Nx, Ny = 1024, 1024
Lx, Ly = 128.0, 128.0
dx, dy = Lx / Nx, Ly / Ny

xg = xp.arange(Nx, dtype=xp.float64) * dx
yg = xp.arange(Ny, dtype=xp.float64) * dy

dt         = 0.003
nsteps     = 10000              # Total times the time-step is run
save_every = 200                # The amount of loops run before a snapshot is made

# ---------- sponge-layer (For damping on the sides) -----------------------
sponge_w   = 0.12 * Lx          # width sponge
sponge_amp = 2.5                # strength sponge

sponge_1d = xp.zeros_like(xg)

# left sponge: 0 <= x < width
left = xg < sponge_w
xi   = xg[left] / sponge_w          
sponge_1d[left] = sponge_amp * xp.cos(0.5 * np.pi * xi)**2

# right sponge: Lx-width < x <= Lx
right = xg > Lx - sponge_w
xi    = (Lx - xg[right]) / sponge_w 
sponge_1d[right] = sponge_amp * xp.cos(0.5 * np.pi * xi)**2


Sponge = sponge_1d[:, None]      # broadcast to (Nx,Ny)

plt.plot(xg.get(), sponge_1d.get()); plt.ylim(0, );


# -------------------- droplet & bath parameters ----------------------
M    = 0.65
R0   = 0.54
Bo   = 4.12
eps  = 0.18

Gammag = 3.4
Omega0 = 4 * np.pi                 
G      = 4.12

d0          = 13.45                             # Depth at deepest part
d1          = 2.02                              # Depth at shallowest part
slope_len   = 0.40 * Lx
slope_mid   = 0.364453 * Lx
chi         = 1.12 * slope_len
# chi determines the slope: slope = (d0 - d1) / (2 chi) = 0.0797
tanharg     = (xg - slope_mid) / chi            # dimensionless argument
depth_1d    = 0.5*(d0 + d1) + 0.5*(d0 - d1) * xp.tanh(tanharg)

FLUX    = 11.298
D       = depth_1d[:, None]
vB_x    = FLUX / D

d_EH = 5.73                                         # Calculated Event Horizon Depth for given Flow
i_EH   = np.argmin(np.abs(depth_1d.get() - d_EH))   # nearest grid pt
x_EH   = i_EH * dx

nu4   = 1e-8
C_air = 0.0022

c1, c2, c3, c4 = 0.7, 8, 0.7, 0.13
alpha = np.pi * eps * R0 * c2
beta  = 2 * np.pi * 4.12

# -------------------- spectral infrastructure ------------------------
kx = 2*np.pi * cfft.fftfreq(Nx, d=dx)[:, None]
ky = 2*np.pi * cfft.fftfreq(Ny, d=dy)[None, :]
K2 = kx**2 + ky**2
k_mag = xp.sqrt(K2)
k_mag[0, 0] = 1e-12                  # avoid divide-by-zero

k_cut  = (2/3) * k_mag.max()
dealias = (k_mag <= k_cut)

# -------------------- field containers  -------------------------
phi_hat      = xp.zeros((Nx, Ny), dtype=xp.complex128)
eta_hat      = xp.zeros_like(phi_hat)
phi_bar_hat  = xp.zeros_like(phi_hat)
eta_bar_hat  = xp.zeros_like(phi_hat)
P_D_hat      = xp.zeros_like(phi_hat)

# -------------------- droplet state  ----------------------------
q = np.array([0.55*Lx, Ly/2], dtype=float)      # horizontal position (x,y)
u = np.array([-3.16, 0.0])                      # horizontal velocity
z = 0.0                                         # vertical position
w = 0.0                                         # vertical velocity
in_contact = False

tol      = 0.01 * R0
R_sigma  = 0.4  * R0

traj      = np.empty((nsteps + 1, 2))
traj[0]   = q
snapshots = []

# -------------------- helper methods: Gaussian impact pressure ---------------
def pressure_gaussian(q_xy_cpu, N):
    """Return GPU FFT of the Gaussian impact pressure centred at q_xy."""
    if N <= 0.0:
        return xp.zeros_like(P_D_hat)

    q_xy = xp.asarray(q_xy_cpu, dtype=xp.float64)

    dx_grid = ((xg[:, None] - q_xy[0] + 0.5*Lx) % Lx) - 0.5*Lx
    dy_grid = ((yg[None, :] - q_xy[1] + 0.5*Ly) % Ly) - 0.5*Ly
    r2      = dx_grid**2 + dy_grid**2

    P_real = (N / (2*np.pi*R_sigma**2)) * xp.exp(-r2 / (2*R_sigma**2))
    return cfft.fft2(P_real)

# -------------------- advective contribution in the PDE ---------------
def advect_x(field_hat):
    field_x_real = cfft.ifft2(1j * kx * field_hat).real   
    return cfft.fft2(vB_x * field_x_real)                

# -------------------- sponge function ---------------
def sponge(field_hat):
    real = cfft.ifft2(field_hat).real       
    return -cfft.fft2(Sponge * real)       

# -------------------- right-hand side for RK-4 ------------------------
def rhs_hat(phi_h, eta_h, phi_bar_h, eta_bar_h, Ga_t):

    adv_phi = advect_x(phi_h)
    adv_eta = advect_x(eta_h)
    adv_phi_bar = advect_x(phi_bar_h)
    adv_eta_bar = advect_x(eta_bar_h)

    dphi =  (Ga_t - Bo*K2) * eta_h \
          - 2*eps*K2 * phi_h       \
          - P_D_hat                \
          - adv_phi                \
          - nu4*K2**2 * phi_h      \
          + sponge(phi_h)

    deta =  k_mag * xp.tanh(k_mag*d0) * phi_h \
          - 2*eps*K2 * eta_h                  \
          - adv_eta                           \
          - nu4*K2**2 * eta_h                 \
          + sponge(eta_h)

    dphi_bar = (Ga_t - Bo*K2) * eta_bar_h \
               - 2*eps*K2 * phi_bar_h     \
               - adv_phi_bar              \
               - nu4*K2**2 * phi_bar_h    \
               + sponge(phi_bar_h)

    deta_bar = k_mag * xp.tanh(k_mag*d0) * phi_bar_h  \
               - 2*eps*K2 * eta_bar_h                 \
               - adv_eta_bar                          \
               - nu4*K2**2 * eta_bar_h                \
               + sponge(eta_bar_h)

    return dphi, deta, dphi_bar, deta_bar

# -------------------- main time-stepping loop ------------------------
eta_prev, eta_bar_prev = 0.0, 0.0

for n in range(nsteps):
    t = n * dt
    i = int(q[0] / dx) % Nx
    j = int(q[1] / dy) % Ny

    # --- local surface values at droplet position -------------------
    eta_real_gpu      = cfft.ifft2(eta_hat).real
    eta_bar_real_gpu  = cfft.ifft2(eta_bar_hat).real

    eta_local      = float(eta_real_gpu[i, j].get())
    eta_bar_local  = float(eta_bar_real_gpu[i, j].get())

    d_eta_dt      = (eta_local     - eta_prev)     / dt
    d_eta_bar_dt  = (eta_bar_local - eta_bar_prev) / dt
    eta_prev, eta_bar_prev = eta_local, eta_bar_local

    # --- bath acceleration g(t) -------------------------------------
    Ga_t = -G*(1 - Gammag * np.cos(Omega0 * t) )

    # --- contact <--> flight switching ---------------------------------
    gap     = z - eta_local
    gap_bar = z - eta_bar_local
    impact  = (gap <= tol)    and (w - d_eta_dt < 0)
    takeoff = (gap_bar >= 1*tol) and (w - d_eta_bar_dt > 0)

    if in_contact and takeoff:
        in_contact = False
        phi_bar_hat.fill(0.0), eta_bar_hat.fill(0.0)
    elif (not in_contact) and impact:
        in_contact = True
        phi_bar_hat[:] = phi_hat
        eta_bar_hat[:] = eta_hat

    # --- vertical dynamics & normal force ---------------------------
    if in_contact:
        dz    = max(z - eta_bar_local, 1e-8)
        const = np.log(abs(c1*R0 / dz))
        const = np.sign(const) * max(abs(const), 1e-4)

        dwdt = (1 / (M*(1 + c3/const**2))) * (
                 M*Ga_t
               - alpha/const * (w - d_eta_bar_dt)
               - beta /const * (z - eta_bar_local)
               )
        N = max(0.0, M*dwdt - M*Ga_t)
    else:
        dwdt, N = Ga_t, 0.0

    # --- impact pressure  --------------------------------------
    if in_contact:
        P_D_hat[:] = pressure_gaussian(q, N)   # copy the Gaussian into the array
    else:
        P_D_hat.fill(0.0)                      # zero it in place


    # --- RK-4 integration of the fields -----------------------------
    Ga1 = Ga_t
    Ga2 = -G + Gammag * np.cos(Omega0 * (t + 0.5*dt))
    Ga3 = Ga2
    Ga4 = -G + Gammag * np.cos(Omega0 * (t + dt))

    k1phi, k1eta, k1phib, k1etab = rhs_hat(phi_hat, eta_hat, phi_bar_hat, eta_bar_hat, Ga1)
    k2phi, k2eta, k2phib, k2etab = rhs_hat(phi_hat + 0.5*dt*k1phi,   eta_hat + 0.5*dt*k1eta,
                                   phi_bar_hat + 0.5*dt*k1phib, eta_bar_hat + 0.5*dt*k1etab, Ga2)
    k3phi, k3eta, k3phib, k3etab = rhs_hat(phi_hat + 0.5*dt*k2phi,   eta_hat + 0.5*dt*k2eta,
                                   phi_bar_hat + 0.5*dt*k2phib, eta_bar_hat + 0.5*dt*k2etab, Ga3)
    k4phi, k4eta, k4phib, k4etab = rhs_hat(phi_hat + dt*k3phi,       eta_hat + dt*k3eta,
                                   phi_bar_hat + dt*k3phib,   eta_bar_hat + dt*k3etab, Ga4)

    phi_hat      += dt/6 * (k1phi + 2*k2phi + 2*k3phi + k4phi)
    eta_hat      += dt/6 * (k1eta + 2*k2eta + 2*k3eta + k4eta)
    phi_bar_hat  += dt/6 * (k1phib + 2*k2phib + 2*k3phib + k4phib)
    eta_bar_hat  += dt/6 * (k1etab + 2*k2etab + 2*k3etab + k4etab)

    # de-alias
    phi_hat     *= dealias
    eta_hat     *= dealias
    phi_bar_hat *= dealias
    eta_bar_hat *= dealias

    # --- update droplet motion --------------------------------
    w += dt*dwdt
    z += dt*w

    if in_contact:
        phase = xp.exp(-1j*(kx*q[0] + ky*q[1]))          # shift origin to the drop
        eta_mem_hat = eta_hat - eta_bar_hat              # memory part only

        grad_eta_gpu = xp.stack([
                cfft.ifft2(1j*kx*eta_mem_hat*phase).real,
                cfft.ifft2(1j*ky*eta_mem_hat*phase).real
            ])[:, 0, 0]                                   # value at (qx,qy)

        grad_eta   = xp.asnumpy(grad_eta_gpu)
        u += (dt/M)*(-(N*c4*np.sqrt(R0/Bo)+C_air)*u - N*grad_eta)
    else:
        u += dt * (-C_air/M) * u

    # periodic BCs for the droplet
    q += dt*u
    q[0] %= Lx
    q[1] %= Ly

    # store diagnostics ----------------------------------------------
    traj[n+1] = q
    if (n+1) % save_every == 0:
        snapshots.append(xp.asnumpy(eta_real_gpu))
        print(f"step {n+1}/{nsteps} saved  (t = {t+dt:.3f})")

# --------------------  output -----------------------------------
np.save("eta_snapshots.npy",
        np.asarray(snapshots, dtype=np.float32))   
np.savetxt("trajectory_xy.csv", traj, delimiter=",", header="x,y")
np.save("event_horizon_x.npy", np.array([i_EH], dtype=np.int32))
print("Simulation complete : snapshots and trajectory written.")

In [ ]:
'''
Run this code after the PDE solver, it will turn the snapshots into a fully
working animation.
'''
from __future__ import annotations
import sys, pathlib, numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim


def _cli_positional() -> list[str]:
    return [tok for tok in sys.argv[1:] if not tok.startswith('-')]

pos = _cli_positional()
IN_PATH  = 'eta_snapshots.npy'
OUT_PATH = 'wavefield.mp4'
FPS      = int(pos[2])           if len(pos) > 2 else 20
try:
    i_EH = int(np.load("event_horizon_x.npy")[0])
except FileNotFoundError:
    i_EH = None      # fallback: no line will be drawn


def _load_snapshot_stack(path: pathlib.Path) -> np.ndarray:
    try:
        raw = np.load(path)
    except (OSError, FileNotFoundError) as e:
        sys.exit(f"Error: {e}")

    if raw.dtype == object:
        try:
            eta = np.stack(raw).astype(np.float32)
        except Exception as e:
            sys.exit(f"Error: could not convert object array to numeric tensor → {e}")
    else:
        eta = raw.astype(np.float32, copy=False)
    return eta

eta = _load_snapshot_stack(IN_PATH)
N_FRAMES, NX, NY = eta.shape
print(f"Loaded {N_FRAMES} frames ({NX}×{NY}).  Building animation …")


fig, ax = plt.subplots(figsize=(6.4, 5.0))
im = ax.imshow(eta[0].T, cmap='PuBu', origin='lower', extent=[0, NX, 0, NY], animated=True)
if i_EH is not None:                         # draw the horizon
    ax.axvline(i_EH, color='0.4', lw=2, ls="--",
               label="event horizon")
ax.set_xlabel('x')
ax.set_ylabel('y')
if i_EH is not None:
    ax.legend(loc="upper right")
fig.colorbar(im, ax=ax, label="η")


def _update(frame: int):
    im.set_array(eta[frame].T)
    ax.set_title(f"wave field – frame {frame}/{N_FRAMES-1}")
    if (frame) % 8 == 0:
      fig.savefig(f"frame {frame}")
    return (im,)

writer = anim.FFMpegWriter(fps=FPS, bitrate=2000)
ani = anim.FuncAnimation(fig, _update, frames=N_FRAMES, blit=True)

try:
    ani.save(OUT_PATH, writer=writer)
except Exception as e:
    sys.exit(f"Error during save: {e}")

# New section